In [11]:
import numpy as np 

In [17]:
from itertools import permutations

class NumberBoxGame():
    OPERATORS = ['+', '-', '*', '/']

    def __init__(self, number_of_numbers, range_of_numbers=10, range_of_target=20):
        
        self.number_of_numbers = number_of_numbers

        self.range_of_numbers = range_of_numbers
        self.range_of_target = range_of_target

        self.solver = {
            '+': lambda x, y: x + y,
            '-': lambda x, y: x - y,
            '*': lambda x, y: x * y,
            '/': lambda x, y: x / y
        }

        self.numbers = []
        self.target = None
        self.solution = ([], [])


    def generate_numbers(self, init=False):

        print('making the game') if not init else None
        self.numbers = np.random.randint(1, self.range_of_numbers, size=self.number_of_numbers)
        self.numbers = sorted(self.numbers, reverse=False)
        self.target = np.random.randint(1, self.range_of_target)

        while not self.find_solution_brute_force():
            self.target = np.random.randint(1, self.range_of_target)
        
        print('game made!') if not init else None


    def _print_numbers(self):
        print(f"Numbers: {', '.join([str(num) for num in self.numbers])}")
        print("Target: ", self.target)
        print()

    def _parse_no_spaces(self, answer:str):
        '''
        read the answer, parse to a list of numbers and operators
        '''

        answer_as_list = []
        i = 0

        while answer != "":
            if answer[i].isdigit():
                # add to intermediate string
                number_term = answer[i]
                # if number is more than one digit
                while i < len(answer) - 1 and answer[i+1].isdigit():
                    number_term += answer[i+1]
                    i += 1
                answer_as_list.append(number_term)
                answer = answer[i+1:]
                i = 0

            elif answer[i] in NumberBoxGame.OPERATORS:
                answer_as_list.append(answer[i])
                answer = answer[i+1:]
                i = 0
            else:
                print(f"Answer contains invalid character: ({answer[i]})." )
                return None

        return answer_as_list

    def _parse_answer(self, answer:str):
        '''
        read the answer, parse to a formula
        '''
        numbers_ans = []
        operators_ans = []

        print("This is your solution: ", answer)

        answer_as_list = answer.split()
        if len(answer_as_list) == 1:
            # no spaces are used
            answer_as_list = self._parse_no_spaces(answer)

        for term in answer_as_list:
            if term in NumberBoxGame.OPERATORS:
                operators_ans.append(term)
            else:
                numbers_ans.append(int(term))
        
        return (numbers_ans, operators_ans)
    

    def _is_answer_valid(self, numbers_sol, operators_sol):
        '''
        check if the solution is valid:
        - all numbers are used
        - numbers are used once
        - operators are used once
        ''' 

        if len(numbers_sol) != self.number_of_numbers:
            message = f"Invalid solution! You have to use exactly the ({self.number_of_numbers}) numbers."
            return False, message
        if len(operators_sol) != self.number_of_numbers - 1:
            message = f"Invalid solution! You have to use exactly ({self.number_of_numbers - 1}) operators."
            return False, message
        for num in self.numbers:
            if num not in numbers_sol:
                message = f"Invalid solution! You have to use all numbers. You are missing ({num})."
                return False, message
        for num in numbers_sol:
            if num not in self.numbers:
                message = f"Invalid solution! You used a number that is not in the available. You used ({num})."
                return False, message
        if len(set(operators_sol)) != len(operators_sol):
            message = f"Invalid solution! You used an operator more than once."
            return False, message
        
        return (True, "")
             

    def solve_this_combination(self, numbers_sol, operators_sol):
        result = numbers_sol[0]

        for operator, number in zip(operators_sol, numbers_sol[1:]):
            result = self.solver[operator](result, number)

        return result
    

    def check_user_answer(self, answer:str):
        '''
        check if the answer is correct
        '''
        numbers_ans, operators_ans = self._parse_answer(answer)

        valid, message = self._is_answer_valid(numbers_ans, operators_ans)
        if not valid:
            print(message)
            return False

        result = self.solve_this_combination(numbers_ans, operators_ans)

        self.show_solution(numbers_ans, operators_ans)
        
        if result == self.target:
            return True
        else:
            return False
        
        
    def let_user_answer(self):

        solution = ""
        solution = input("Enter your solution: ")
        
        if solution == "":
            return False
        elif solution == "-q":
            return True
        else:
            pass

        if self.check_user_answer(solution):
            print("Correct!")
            return True
        else:
            print("Incorrect!")
            return False


    def show_solution(self, numbers_sol=None, operators_sol=None):
        ''''''
    
        solution = ""

        old_res = numbers_sol[0]

        for number, operator in zip(numbers_sol[1:], operators_sol):

            result = self.solver[operator](old_res, number)
            
            solution += f"{old_res} {operator} {number} = {result}\n"
            old_res = result

        solution += f"Final result:\n{result}"

        print(solution)

    def play_round(self):
        
        self.generate_numbers()
        self._print_numbers()

        correct = False
        attempts = 0

        while not correct and attempts < 3:
            correct = self.let_user_answer()
            attempts += 1

        print()
        if correct:
            print("You win!")
        else:
            print("You lose!")

        print("This is our solution:")
        self.show_solution(*self.solution)


    def find_solution_brute_force(self):
        '''
        find a solution using brute force
        '''
        # find all possible permutations
        num_permutations_list = list(permutations(self.numbers))
        # find all possible combinations of operators
        operator_combinations = list(permutations(NumberBoxGame.OPERATORS, self.number_of_numbers - 1))

        # print(f"Number of permutations: {len(num_permutations_list)}")
        # print(f"Number of operator combinations: {len(operator_combinations)}")
        # print(f"Total number of combinations: {len(num_permutations_list) * len(operator_combinations)}")

        counter = 0

        for num_combi in num_permutations_list:
            for op_combi in operator_combinations:
                counter += 1
                result = self.solve_this_combination(num_combi, op_combi)
                if result == self.target:
                    print(f"Found a solution! Target: {self.target}, Result: {result}")
                    self.solution = (num_combi, op_combi)
                    return True
                else:
                    pass

        return False

In [19]:
game = NumberBoxGame(number_of_numbers=4, 
                     range_of_numbers=20, 
                     range_of_target=20)

print('New game!')
game.play_round()


New game!
making the game
Found a solution! Target: 8, Result: 8.0
game made!
Numbers: 3, 7, 7, 10
Target:  8

This is your solution:  7/7+10-3
7 / 7 = 1.0
1.0 + 10 = 11.0
11.0 - 3 = 8.0
Final result:
8.0
Correct!

You win!
This is our solution:
7 / 7 = 1.0
1.0 - 3 = -2.0
-2.0 + 10 = 8.0
Final result:
8.0
